In [2]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

# Define a custom dataset class for generating poems
class PoemDataset(Dataset):
    def __init__(self, poems, tokenizer, max_length=100):
        self.poems = poems
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.poems)

    def __getitem__(self, idx):
        poem = self.poems[idx]
        input_ids = self.tokenizer.encode(poem, max_length=self.max_length, truncation=True)
        return torch.tensor(input_ids, dtype=torch.long)

# Load poems from a .txt file
def load_poems_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        poems = file.read().split('\n\n')  # Assuming poems are separated by empty lines
    return poems

# Define path to the .txt file containing poems
txt_file_path = 'poem.txt'

# Load poems from the .txt file
poems = load_poems_from_txt(txt_file_path)

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define training parameters
num_epochs = 20
batch_size = 4
max_length = 200

# Create dataset and data loader
dataset = PoemDataset(poems, tokenizer, max_length=max_length)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode

    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch.to(model.device)
        outputs = model(input_ids=input_ids, labels=input_ids)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Save trained model checkpoint
torch.save(model.state_dict(), 'poem_generator_model.pth')


Epoch 1/20, Loss: 5.118703842163086
Epoch 2/20, Loss: 5.061988353729248
Epoch 3/20, Loss: 2.975680112838745
Epoch 4/20, Loss: 2.0558178424835205
Epoch 5/20, Loss: 1.207516074180603
Epoch 6/20, Loss: 0.9351003170013428
Epoch 7/20, Loss: 0.6878919005393982
Epoch 8/20, Loss: 0.4367513656616211
Epoch 9/20, Loss: 0.3448368310928345
Epoch 10/20, Loss: 0.2254834622144699
Epoch 11/20, Loss: 0.13153383135795593
Epoch 12/20, Loss: 0.11354824900627136
Epoch 13/20, Loss: 0.10814617574214935
Epoch 14/20, Loss: 0.05989956110715866
Epoch 15/20, Loss: 0.03741941601037979
Epoch 16/20, Loss: 0.11595889180898666
Epoch 17/20, Loss: 0.03822680190205574
Epoch 18/20, Loss: 0.0908135250210762
Epoch 19/20, Loss: 0.05846743285655975
Epoch 20/20, Loss: 0.12713192403316498


In [3]:
!pip install transformers
!pip install modin
!pip uninstall torch
!pip install torch==1.13.1
!pip install intel_extension_for_pytorch==1.13.100

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 35.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.20.1 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.4 which is incompatible.
Found existing i

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 5.4 MB/s eta 0:00:00


In [7]:
!pip uninstall torch
!pip install torch==1.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.13.0 which is incompatible.


## **Training with/without IPEX (Intel Extension for pytorch)**

### Without IPEX

In [14]:
import torch
import torch.nn as nn
import time

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define training parameters
num_epochs = 10
learning_rate = 0.001
batch_size = 32
input_size = 10
output_size = 1

# Generate random input and output data
x_train = torch.randn(1000, input_size)
y_train = torch.randn(1000, output_size)

# Create the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop with optim
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Optim Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Total training time
print(f"Total Training Time: {time.time() - start_time} seconds")


Optim Epoch [1/10], Loss: 2.0920748710632324
Optim Epoch [2/10], Loss: 1.9652900695800781
Optim Epoch [3/10], Loss: 1.853972315788269
Optim Epoch [4/10], Loss: 1.7561496496200562
Optim Epoch [5/10], Loss: 1.6701105833053589
Optim Epoch [6/10], Loss: 1.5943677425384521
Optim Epoch [7/10], Loss: 1.5276281833648682
Optim Epoch [8/10], Loss: 1.4687671661376953
Optim Epoch [9/10], Loss: 1.4168058633804321
Optim Epoch [10/10], Loss: 1.3708926439285278
Total Training Time: 0.40898799896240234 seconds


In [16]:
import torch
import torch.nn as nn
import time

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define training parameters
num_epochs = 20
learning_rate = 0.001
batch_size = 32
input_size = 10
output_size = 1

# Generate random input and output data
x_train = torch.randn(1000, input_size)
y_train = torch.randn(1000, output_size)

# Create the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop with optim
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Optim Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Total training time
print(f"Total Training Time: {time.time() - start_time} seconds")


Optim Epoch [1/20], Loss: 0.9241153001785278
Optim Epoch [2/20], Loss: 0.8671150803565979
Optim Epoch [3/20], Loss: 0.8195785284042358
Optim Epoch [4/20], Loss: 0.7801409959793091
Optim Epoch [5/20], Loss: 0.7476246953010559
Optim Epoch [6/20], Loss: 0.7210130095481873
Optim Epoch [7/20], Loss: 0.6994288563728333
Optim Epoch [8/20], Loss: 0.6821167469024658
Optim Epoch [9/20], Loss: 0.6684258580207825
Optim Epoch [10/20], Loss: 0.6577962040901184
Optim Epoch [11/20], Loss: 0.6497465968132019
Optim Epoch [12/20], Loss: 0.6438636183738708
Optim Epoch [13/20], Loss: 0.639792799949646
Optim Epoch [14/20], Loss: 0.637230396270752
Optim Epoch [15/20], Loss: 0.6359168291091919
Optim Epoch [16/20], Loss: 0.6356302499771118
Optim Epoch [17/20], Loss: 0.6361813545227051
Optim Epoch [18/20], Loss: 0.6374095678329468
Optim Epoch [19/20], Loss: 0.6391780376434326
Optim Epoch [20/20], Loss: 0.6413711309432983
Total Training Time: 0.6625721454620361 seconds


In [33]:
import torch
import torch.nn as nn
import time

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define training parameters
num_epochs = 40
learning_rate = 0.001
batch_size = 32
input_size = 10
output_size = 1

# Generate random input and output data
x_train = torch.randn(1000, input_size)
y_train = torch.randn(1000, output_size)

# Create the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop with optim
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Optim Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Total training time
print(f"Total Training Time: {time.time() - start_time} seconds")


Optim Epoch [1/40], Loss: 0.86285400390625
Optim Epoch [2/40], Loss: 0.8515642881393433
Optim Epoch [3/40], Loss: 0.8436882495880127
Optim Epoch [4/40], Loss: 0.8386602401733398
Optim Epoch [5/40], Loss: 0.835996150970459
Optim Epoch [6/40], Loss: 0.8352826833724976
Optim Epoch [7/40], Loss: 0.8361666798591614
Optim Epoch [8/40], Loss: 0.8383474946022034
Optim Epoch [9/40], Loss: 0.8415696024894714
Optim Epoch [10/40], Loss: 0.8456161618232727
Optim Epoch [11/40], Loss: 0.8503037095069885
Optim Epoch [12/40], Loss: 0.855477511882782
Optim Epoch [13/40], Loss: 0.8610072731971741
Optim Epoch [14/40], Loss: 0.866784393787384
Optim Epoch [15/40], Loss: 0.8727173805236816
Optim Epoch [16/40], Loss: 0.8787308931350708
Optim Epoch [17/40], Loss: 0.8847628831863403
Optim Epoch [18/40], Loss: 0.8907622694969177
Optim Epoch [19/40], Loss: 0.8966876268386841
Optim Epoch [20/40], Loss: 0.9025061130523682
Optim Epoch [21/40], Loss: 0.9081912636756897
Optim Epoch [22/40], Loss: 0.9137226343154907
Op

### With Ipex

In [15]:
import torch
import torch.nn as nn
import time
import intel_extension_for_pytorch as ipex

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define training parameters
num_epochs = 10
learning_rate = 0.001
batch_size = 32
input_size = 10
output_size = 1

# Generate random input and output data
x_train = torch.randn(1000, input_size)
y_train = torch.randn(1000, output_size)

# Create the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Optimize the model with IPEX, passing the optimizer
model, optimizer = ipex.optimize(model, optimizer=optimizer)

# Training loop
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Optim Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Total training time
print(f"Total Training Time: {time.time() - start_time} seconds")


Optim Epoch [1/10], Loss: 0.8854290246963501
Optim Epoch [2/10], Loss: 0.8616810441017151
Optim Epoch [3/10], Loss: 0.8404115438461304
Optim Epoch [4/10], Loss: 0.821343183517456
Optim Epoch [5/10], Loss: 0.8042304515838623
Optim Epoch [6/10], Loss: 0.7888562083244324
Optim Epoch [7/10], Loss: 0.7750279307365417
Optim Epoch [8/10], Loss: 0.7625757455825806
Optim Epoch [9/10], Loss: 0.7513487935066223
Optim Epoch [10/10], Loss: 0.7412134408950806
Total Training Time: 0.15320825576782227 seconds


In [19]:
import torch
import torch.nn as nn
import time
import intel_extension_for_pytorch as ipex

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define training parameters
num_epochs = 20
learning_rate = 0.001
batch_size = 32
input_size = 10
output_size = 1

# Generate random input and output data
x_train = torch.randn(1000, input_size)
y_train = torch.randn(1000, output_size)

# Create the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Optimize the model with IPEX, passing the optimizer
model, optimizer = ipex.optimize(model, optimizer=optimizer)

# Training loop
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Optim Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Total training time
print(f"Total Training Time: {time.time() - start_time} seconds")


Optim Epoch [1/20], Loss: 2.200347423553467
Optim Epoch [2/20], Loss: 2.1515400409698486
Optim Epoch [3/20], Loss: 2.107692003250122
Optim Epoch [4/20], Loss: 2.068255662918091
Optim Epoch [5/20], Loss: 2.032745838165283
Optim Epoch [6/20], Loss: 2.0007340908050537
Optim Epoch [7/20], Loss: 1.9718413352966309
Optim Epoch [8/20], Loss: 1.9457323551177979
Optim Epoch [9/20], Loss: 1.9221101999282837
Optim Epoch [10/20], Loss: 1.9007114171981812
Optim Epoch [11/20], Loss: 1.8813029527664185
Optim Epoch [12/20], Loss: 1.863677978515625
Optim Epoch [13/20], Loss: 1.8476520776748657
Optim Epoch [14/20], Loss: 1.8330620527267456
Optim Epoch [15/20], Loss: 1.819762945175171
Optim Epoch [16/20], Loss: 1.8076251745224
Optim Epoch [17/20], Loss: 1.7965340614318848
Optim Epoch [18/20], Loss: 1.7863866090774536
Optim Epoch [19/20], Loss: 1.7770910263061523
Optim Epoch [20/20], Loss: 1.7685662508010864
Total Training Time: 0.2639493942260742 seconds


In [26]:
import torch
import torch.nn as nn
import time
import intel_extension_for_pytorch as ipex

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define training parameters
num_epochs = 40
learning_rate = 0.001
batch_size = 32
input_size = 10
output_size = 1

# Generate random input and output data
x_train = torch.randn(1000, input_size)
y_train = torch.randn(1000, output_size)

# Create the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Optimize the model with IPEX, passing the optimizer
model, optimizer = ipex.optimize(model, optimizer=optimizer)

# Training loop
start_time = time.time()
for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Optim Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Total training time
print(f"Total Training Time: {time.time() - start_time} seconds")


Optim Epoch [1/40], Loss: 1.0372893810272217
Optim Epoch [2/40], Loss: 0.9968695640563965
Optim Epoch [3/40], Loss: 0.9627225995063782
Optim Epoch [4/40], Loss: 0.9339970350265503
Optim Epoch [5/40], Loss: 0.9099512696266174
Optim Epoch [6/40], Loss: 0.8899393081665039
Optim Epoch [7/40], Loss: 0.8733994364738464
Optim Epoch [8/40], Loss: 0.8598429560661316
Optim Epoch [9/40], Loss: 0.8488450050354004
Optim Epoch [10/40], Loss: 0.8400364518165588
Optim Epoch [11/40], Loss: 0.8330973982810974
Optim Epoch [12/40], Loss: 0.827750027179718
Optim Epoch [13/40], Loss: 0.8237534761428833
Optim Epoch [14/40], Loss: 0.820899486541748
Optim Epoch [15/40], Loss: 0.8190079927444458
Optim Epoch [16/40], Loss: 0.817923367023468
Optim Epoch [17/40], Loss: 0.8175113797187805
Optim Epoch [18/40], Loss: 0.8176562786102295
Optim Epoch [19/40], Loss: 0.8182589411735535
Optim Epoch [20/40], Loss: 0.819233775138855
Optim Epoch [21/40], Loss: 0.820507824420929
Optim Epoch [22/40], Loss: 0.8220189213752747
Op